In [3]:
import os, pdfplumber, pprint
from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from itertools import chain
from dotenv import load_dotenv

In [4]:
load_dotenv('../.env')
openai.api_key = os.getenv("CHATGPT-RECEIPT")

In [5]:
client = OpenAI(api_key=os.getenv("CHATGPT-RECEIPT"))

In [9]:
file = client.files.create(
    file = open('../pdf/r-d-innovation.pdf', 'rb'),
    purpose = 'assistants'
)

vector_store = client.beta.vector_stores.create(
    name="R&D Innovation"
)

# 파일을 Vector Store에 추가
client.beta.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file.id
)

VectorStoreFile(id='file-HZUR1boj7a8yoeT5aqudr4', created_at=1735477631, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_4Tlnuv9bY64mEL6I2349JUc8', chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [28]:
# Assistant 생성
assistant = client.beta.assistants.create(
    name = "My Assistant",
    instructions = """
                넌 국가연구개발 혁신법에 대해 잘 아는 행정 관련 전문가야.
                만약 Q&A에 해당 내용이 있으면 이에 대해 자세하고 친절하게 답변해줘.
                정확하다고 생각되는 답을 찾았으면, 가장 먼저 "예" 혹은 "아니오"로 답변을 시작하고, 그렇지 않으면 "모르겠어"라고 말해줘.
                답한 내용의 근거를 정확하게 말해주되 문서내 검색한 내용만 답해 줘.
                마침표 다음에 개행 문자를 넣어줘.
    """,
    model="gpt-4-turbo-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"}
    ],
    tool_resources={
        "file_search": {"vector_store_ids": [vector_store.id]}
    }
)


In [30]:
thread.id

'thread_tN6512mlmOREUL8wiBHMm9kV'

In [31]:
# Thread 생성
thread = client.beta.threads.create()
print(f'thread id = {thread.id}')
# 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="공동연구기관 변경을 하기 위해서는 어떤 절차를 밟아야 하나요?"
)

# Run 생성 및 실행
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Run 완료 대기
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# 결과 메시지 가져오기
messages = client.beta.threads.messages.list(thread_id=thread.id)
# 결과 출력
for message in messages:
    if message.role == "assistant":
        print(message.content[0].text.value)


thread id = thread_teNuvGoZTZRt0ihKfLQC4uZV
공동연구개발기관 변경을 하기 위해서는 중요한 협약 변경에 해당하는 사항입니다. 이에 따라, 협약 당사자 간 상호협의(실무 상 승인)를 거쳐 변경해야 합니다【4:8†source】. 또한, 혁신법 제11조(연구개발과제 협약 등) 제2항에 따르면, 연구개발기관을 추가하거나 변경하거나 연구책임자, 연구개발 목표, 연구개발비, 연구개발기간 등 연구개발과제 수행에 관한 중요한 사항을 변경할 필요가 있을 때에는 대통령령으로 정하는 바에 따라 협의하여 해당 연구개발과제 협약의 내용을 변경할 수 있습니다 .

따라서, 공동연구개발기관을 변경하고자 하는 경우, 이러한 절차에 따라 중앙행정기관의 장과 협의하여 변경해야 함을 알 수 있습니다.


In [43]:
print(f'thread id = {thread.id}')
# 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        {
            "상호명": "고향옥",
            "날짜": "2024-12-13",
            "항목": [
                {"이름": "세트 A", "가격": 13000},
                {"이름": "세트 B", "가격": 12000}
            ],
            "총액": 25000
        }
        이런 영수증 내역이 있을 때, 연구비 어떤 항목으로 처리해야 하나요?
        상호를 검색해서 업종을 판단해 주세요.
        단답형으로 항목만 답변해 주세요. (ex, 출장비)
        정확한 항목이 없을 경우, "판단 필요"라고 답변해 주세요.
        판단이 필요한 경우에는 이유를 말해주세요.
    """
)

# Run 생성 및 실행
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Run 완료 대기
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# 결과 메시지 가져오기
messages = client.beta.threads.messages.list(thread_id=thread.id)
# 결과 출력
for message in messages:
    if message.role == "assistant":
        print(message.content[0].text.value)

thread id = thread_teNuvGoZTZRt0ihKfLQC4uZV
판단 필요

인터넷 검색 기능이 제한되어 있어 "고향옥" 상호의 업종을 직접 검색할 수 없습니다. 영수증 내역의 항목인 "세트 A"와 "세트 B"만으로는 연구비 처리를 위한 구체적인 항목(예: 식비, 회의비 등)을 결정하기에 충분한 정보가 아니므로, 상호명으로 업종을 유추할 수 있는 인터넷 검색이나 추가적인 정보가 필요합니다. 따라서, 영수증 내역이 연구비 어떤 항목으로 처리될 수 있는지 판단하기 위해서는 해당 영수증의 사용 목적과 연구비 사용 기준을 확인할 필요가 있습니다.
판단 필요
판단 필요
영수증 내역에 따르면, '주차요금'이 13,000원이고 '할인요금'으로 12,000원이 적용되어 '총액'이 1,000원으로 나타나 있습니다. 이러한 경비는 연구활동과 관련하여 발생한 것이라면 연구비에서 처리할 수 있습니다.

연구활동과 관련된 비용으로 처리할 때는 다음과 같은 항목으로 고려해볼 수 있습니다:

1. **여비(교통비)**: 연구활동을 위한 현장 방문, 데이터 수집, 회의 참석 등을 위해 발생한 교통비용으로 처리할 수 있습니다. 이 경우, '주차요금'이 해당될 수 있습니다.

2. **기타 운영비**: 연구 프로젝트 운영에 필요한 다양한 비용 중 하나로 처리할 수 있으며, 개별 연구과제의 예산 지침에 따라 다를 수 있습니다.

'할인요금'은 주차요금에서 할인된 금액을 의미하며, 실제 연구 프로젝트 예산에서 지출된 금액은 '총액' 1,000원입니다. 따라서, 실질적으로 연구비로 처리해야 할 금액은 1,000원이며, 이는 연구활동과밀접하게 관련된 필요경비로 간주될 수 있습니다.

연구비 사용에 있어 중요한 것은 해당 경비가 연구활동과 직접적으로 관련이 있으며, 연구과제의 예산 집행 규정 및 기준에 일치하는지 확인하는 것입니다. 따라서, 구체적인 처리 방법은 연구과제를 주관하는 기관의 연구비 관리 지침 및 국가연구개발혁신법 등 관련 법령과 지침에 따라야 하며, 필요할 경우 연구비 관

In [14]:
예, 주관기관을 변경하는 경우 승인 사항입니다. 혁신법 제11조제2항에 따르면, 주관기관 변경과 같은 중요한 협약 변경 사항은 상호 협의를 거쳐야 하며, 이는 실무적으로 승인 절차와 동일합니다. 연구책임자 변경이 필요한 경우 특별평가를 실시하여 연구책임자 변경의 승인 여부와 과제 중단 여부 등을 판단할 수 있습니다. 연구개발비 사용계획의 변경 승인 여부를 중앙행정기관의 장은 연구개발기관의 장에게 통보해야 하며, 이는 사용계획 변경 신청 접수 후 15일 이내에 이루어져야 합니다. 다만, 15일 이내에 변경 승인 여부를 결정하기 어려운 경우 기간을 연장할 수 있으며, 연장한 기간을 연구개발기관의 장에게 통보해야 합니다. \n\n경미한 협약 변경 사항인 경우에는 협약 당사자 간의 통보만으로 협약이 변경될 수 있습니다. 예를 들어 연구개발과제의 추진 방법의 변경, 연구개발과제를 수행하는 연구자(연구책임자 제외)의 변경, 연구개발비 사용에 대한 개괄적인 계획의 변경(간접비 및 연구수당 증액 제외), 연구개발기관의 연락처 및 연구지원인력의 변경 등은 경미한 사항으로 통보만으로 협약 변경이 가능합니다.\n\n따라'

"예\n\n주관기관을 변경할 경우, 이는 승인 사항에 해당합니다. 연구개발기관의 장은 변경 사유 및 내용을 중앙행정기관의 장에게 제출하여 승인을 받아야 하며, 중앙행정기관의 장은 변경 승인 여부를 연구개발기관의 장에게 통보해야 합니다. 이는 '국가연구개발혁신법' 매뉴얼에 명시된 내용으로, 주요 협약 변경은 상호간의 협의(실무 상 승인)를 거쳐야 하며, 연구책임자 변경이나 연구개발기관 변경 등의 중요한 변경사항에 대해서는 특별평가를 거칠 수 있습니다.


주관기관을 변경할 때는 승인 사항입니다. 법 제11조제2항에 따라 주관연구개발기관, 공동연구개발기관, 위탁연구개발기관의 변경은 상호협의(승인)이 필요하다고 명시되어 있습니다. 이는 ‘국가연구개발혁신법’ 매뉴얼에 근거하여, 연구개발기관 변경에 관련된 모든 사항(주관, 공동, 위탁연구개발기관)은 사전 협의 및 승인 과정을 거쳐야 한다고 설명되어 있습니다 .

SyntaxError: unexpected character after line continuation character (4084583234.py, line 1)